In [1]:
import os
import pymysql
import pandas as pd
import requests
import time
import urllib3
from bs4 import BeautifulSoup

host = os.getenv('localhost')
port = os.getenv('MYSQL_PORT')
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
database = os.getenv('MYSQL_DATABASE')

conn = pymysql.connect(
    host=host,
    port=int(3306),
    user="root",
    passwd="",
    db="recommender_system",
    charset='utf8mb4')



In [2]:
#PARA BUSCAR UNA PELICULA POR SU CODIGO

def searchMovieIfExist( key ):
    
    sql  = "SELECT * FROM tittle_basics WHERE tconst='%s'"%(key)
   
    df = pd.read_sql_query( sql, conn )
    return df.tail(10)


In [3]:
#searchMovieIfExist("tt0000001")

In [4]:
#OBTIENE TODA LAS LISTAS GURDADAS
def getList(name):
    sql  = "SELECT * FROM lists WHERE name like '%s'"%(name)
   
    df = pd.read_sql_query( sql, conn )
    return df

In [5]:
#getList("nombre prueba")

In [6]:
def insertList(name, link, metadata):
    
    with conn.cursor() as cursor:
        # CREA UNA NUEVA FILA
        sql = "INSERT INTO lists ( name, link, metadata ) VALUES ( %s, %s, %s )"
        cursor.execute( sql,( name, link, metadata ) )
        
        #GUARDAMOS LOS CAMBIOS
        conn.commit()
               
    

In [7]:
#insertList("nombre prueba", "link prueba", "meta prueba")

In [8]:
def getNIterations(number):
    nIteration = 1
    div= number%100
    print ("Resto: ",div)
    if number >= 100:
        
        nIteration = int(number/100)+1
        print("N iteration-> ", nIteration)
        
    return nIteration

In [9]:
def getInfoLists( lists ):
    
    resultList = []
    
    #RECORRO LAS LISTAS Y SACO LA INFORMACIÓN
    for list in lists:
        collectorList = []
        #OBTENGO EL NOMBRE DE LA LISTA
        infoList = list.find( "div" , {"class": "list_name"} )
        nameList = infoList.find( "a" )    
        collectorList.append(nameList.text)


        #OBTENGO EL LINK DE LA LISTA

        linkList = infoList.find( 'a', href = True) 
        collectorList.append(linkList['href'])
        


        #OBTENGO LA METADATA DE LA LISTA
        metadataList = list.find( "div" , {"class": "list_meta"} )
        collectorList.append(metadataList.text)

        resultList.append(collectorList)
        
    return resultList

In [10]:
def formatUrlPage( url, number ):
    url+="?page=%d"%number #url formatada para rec
    return  url
    

In [11]:
def isDownloadable(url):
    """
    Does the url contain a downloadable resource
    """
    h = requests.head(url, allow_redirects=True)
    header = h.headers
    content_type = header.get('content-type')
    content_type= content_type.split('/',1)
    print("Content type "+content_type)
    if 'text' in content_type.lower():
        return False
    if 'html' in content_type.lower():
        return False
    return True

In [12]:
def downloadExcel( urlDomain ,urlIdList ):
    
      time.sleep(1)
    stringExport = "/export?ref_=ttls_otexp"
    url = urlDomain+urlIdList+stringExport
    response = requests.get(url)
    html = response.text()
    print(html)
    
    print ("Downloading "+ url)
    if( isDownloadable (url) ):
        
        idList = urlIdList.split("/",2)[-1]        
        fileName= idList+".csv"
        r = requests.get(url, allow_redirects=True)
        open(fileName, 'wb').write(r.content)       
        len(r.content)
        

In [13]:


def getListImdb(url):
    
    countPage = 1 #indice de la página que debe iterar al menos una vez
    url = formatUrlPage( url, countPage )
    
    r = requests.get(url)
    html_content = r.text
    soup = BeautifulSoup(html_content, 'lxml')
    
    nIterationsText=  soup.find("span",{"class":"list-pagination-total-elements"})
    nIterations = getNIterations( int( nIterationsText.text ) )    
    countIterations = 0
    
    resultList = None 
    countNot200 = 0
    while countIterations < nIterations:
        time.sleep(1.5)
        print("Url: " + url)
        print("CountNot200: ", countNot200)      
            
        if r.status_code == 200:
            countNot200 = 0
            html_content = r.text
            soup = BeautifulSoup(html_content, 'lxml')
            #OBTENGO TODAS LAS LISTAS
            lists = soup.findAll("div", {"class": "list-preview"})
            resultList = getInfoLists( lists )
            countPage+=1
            url=url.split('?',1)[0]
            url = formatUrlPage( url, countPage )
            r = requests.get(url)
            countIterations+=1

        elif countNot200 >= 10:
            
            return resultList
        else:
            countNot200 += 1

    return resultList


In [14]:
#URL DE LA LISTA A BUSCAR
urlDomain='https://www.imdb.com'
urlList="/lists/tt0074486"
url=urlDomain+urlList

#SACAMOS EL CODIGO DE LA PELICULA
tconst=url.split('/',4)[-1]
tconst=tconst.split('?',1)[0]

#limpiamos parametros get
urlSplit= url.split('?',1)[0]

#OBTENEMOS LAS LISTAS
listas = getListImdb(  url) 


  

Resto:  97
N iteration->  10
Url: https://www.imdb.com/lists/tt0074486?page=1
CountNot200:  0
Url: https://www.imdb.com/lists/tt0074486?page=2
CountNot200:  0
Url: https://www.imdb.com/lists/tt0074486?page=2
CountNot200:  1
Url: https://www.imdb.com/lists/tt0074486?page=2
CountNot200:  2
Url: https://www.imdb.com/lists/tt0074486?page=2
CountNot200:  3
Url: https://www.imdb.com/lists/tt0074486?page=2
CountNot200:  4
Url: https://www.imdb.com/lists/tt0074486?page=2
CountNot200:  5
Url: https://www.imdb.com/lists/tt0074486?page=2
CountNot200:  6
Url: https://www.imdb.com/lists/tt0074486?page=2
CountNot200:  7
Url: https://www.imdb.com/lists/tt0074486?page=2
CountNot200:  8
Url: https://www.imdb.com/lists/tt0074486?page=2
CountNot200:  9
Url: https://www.imdb.com/lists/tt0074486?page=2
CountNot200:  10


In [15]:
if isinstance( listas, list):
    
    for lista in listas:
        
        time.sleep(1)
        downloadExcel( urlDomain, lista[1] )    
     #insertList(lista[0], lista[1], lista[2])
else:
    print( "Problema al descargar la data: ", listas)

TypeError: 'str' object is not callable